# Generación de variables aleatorias discretas

## Método de la Transformada Inversa

Dado $X$ una variable aleatoria discreta con función de probabilidad $P(X=x_j)=p_j$, y función de distribución acumulada

$F_X(x) = \begin{cases}
    0 & \text{si } x<x_0\\
    p_0+...+p_{i} & \text{si } x_i\le x< x_{i+1}\\
\end{cases}$,

para $j=0,1,...$, y $n=1,2,...$, generamos $U\thicksim\mathcal{U}(0,1)$ y definimos

$X = \begin{cases}
    x_0 & \text{si } U\in [0,p_0)\\
    x_n & \text{si } U\in [p_0+...+p_{n-1},p_o+...+p_n)\\
\end{cases}.$

De esta manera, $$P(\text{generar }x_0) = P(0 \le U < p_0) = p_0,$$ $$P(\text{generar }x_i) = P(p_0+...+p_{i-1} \le U < p_0+...+p_i) = p_i.$$

In [1]:
from random import random

'''
p: arreglo tal que p[i] = p_i = P(X=i)
x: arreglo de los valores que toma X tal que P(X=x[i]) = P(X=x_i) = p_i
'''
def MetodoTransformadaInversa(p, x):
    U=random()
    i=0; F=p[i]
    while U >= F:
        i += 1; F += p[i]
    return x[i]

#### Ejemplo:
$X$ tome los valores $1,2,3,4$ con probabilidades $0.20,0.15,0.25,0.40$ respectivamente. Entonces, la función de distribución acumulada es $$F_X(x) = \begin{cases}
    0 & \text{si } x<1\\
    0.20 & \text{si } 1\le x<2\\
    0.35 & \text{si } 2\le x<3\\
    0.60 & \text{si } 3\le x<4\\
    1 & \text{si } x\ge 4 \end{cases},$$
y el algoritmo para generar $X$ es el siguiente:

In [13]:
def ejemplo():
    U = random()
    if   U < .20: return 1
    elif U < .35: return 2
    elif U < .60: return 3
    else:         return 4

print(ejemplo())

4


Una forma de optimizar el algoritmo es reduciendo el número de comparaciones. Para ello, se puede ordenar las probabilidades de mayor a menor, y el algoitmo se convierte en:

In [14]:
def ejemploMejorado():
    U = random()
    if   U < .40: return 4
    elif U < .65: return 3
    elif U < .85: return 1
    else:         return 2

print(ejemplo())

3


De esta manera, la media de comparaciones pasará de $$E[\text{número de comparaciones - caso 1}] = 1\cdot 0.20 + 2\cdot 0.15 + 3\cdot 0.65 = 2.45$$ a $$E[\text{número de comparaciones - caso 2}] = 1\cdot 0.40 + 2\cdot 0.25 + 3\cdot 0.35 = 1.45.$$

### Generación de una variable aleatoria uniforme discreta
Si $X$ es una variable con distribución uniforme discreta en $\{1,...,n\}$ entonces $p_1=...=p_n=1/n$, luego la función de distribución acumulada es

$F_X(x) = \begin{cases}
    0 & \text{si } x<1\\
    j/n & \text{si } j\le x < j+1\\
\end{cases}$

Aplicando el método de la inversa, la variable generada $X$ es tal que

$X$
genera
$
j \Leftrightarrow \\
p_0+...+p_{j-1} \le U < p_o+...+p_j \Leftrightarrow \\
(j-1)/n \le U<j/n \Leftrightarrow \\
j-1\le Un<j \Leftrightarrow \\
j-1=\lfloor Un \rfloor\Leftrightarrow \\
j=\lfloor Un \rfloor +1
$

De esta manera, obtenemos una mejora en la eficiencia del algoritmo, ya que se reduce el número de comparaciones, y queda de la siguiente forma

In [ ]:
def Discreta(n):
    return int(n*random()) + 1

Para generar $X$ uniforme en $\{m,m+n-1\}$, se genera $U\thicksim\mathcal{U}(0,1)$ y se define $X=\lfloor Un\rfloor + m$.

Para generar $X$ uniforme, tal que $X\in\{2,4,6,8,10\}$, se genera $U\thicksim\mathcal{U}(0,1)$ y se define $X=2\lfloor 5U\rfloor + 2$.

#### Ejemplo: Generación de permutaciones aleatorias
Dada una lista de $n$ elementos, $a_0,...,a_{n-1}$ se desea generar una permutación aleatoria de la lista con probabilidad $\frac1{N!}$. Para ello, se puede recorrer la lista de derecha a izquierda, y en cada paso $i$ se intercambia el elemento en la posición $i$ con un elemento en una posición aleatoria entre $i$ y $0$. De esta manera, se obtiene una permutación aleatoria de la lista. Y 
$P(\text{generar }(a_0,...,a_{n-1})) =
P(\text{generar }a_{n-1})$ $P(\text{generar }a_{n-2}|a_{n-1})$ $P(\text{generar }a_{n-3}|a_{n-2},a_{n-1})$ ... $P(\text{generar }a_0|a_{n-1},...,a_1) =$ $\frac1{n}\frac1{n-1}...\frac1{1} = \frac1{n!}.$
El algoritmo para generar una permutación aleatoria de una lista de $n$ elementos es el siguiente:

In [19]:
'''
a: arreglo que contiene el orden inicial tal que a[i] = a_i
'''
def Permutar(a):
    for i in range(len(a)-1, 0, -1):
        Indice = int((i+1)*random())
        a[Indice], a[i] = a[i], a[Indice]
    return a

### Cálculo de promedios
Supongamos que se desea calcular un promedio de una gran cantidad de valores:
$$\bar{a}= \frac{1}{N}\sum_{i=1}^{N}a_i = \frac{a_1+...+a_N}{N}.$$
donde $N>>1$ y por la gran cantidad de términos o por la complejidad de éstos resulta muy complicado calcular la suma.
Si definimos $X$ una variable aleatoria uniforme discreta en $[1,N]$, y $g$ una función tal que $g(i)=a_i$, entonces el valor que se desea calcular es $$\bar{a}=E[g(X)].$$
Y finalmente, por la Ley Fuerte de los grandes números, se tiene que $\bar{a}$ se puede estimar como un promedio de M términos de la sumatoria elegidos al azar $$\bar{a}=E[g(X)]\approx\frac{1}{M}\sum_{i=1}^{M}g(X_i),$$ donde $X_i\thicksim X$.

El algoritmo para calcular el promedio de una gran cantidad de valores es el siguiente:

In [20]:
'''
g: Función a promediar
N: Términos de la sumatoria original
M: Términos de la sumatoria de MonteCarlo
'''
def MonteCarlo(g,N,M):
    Suma = 0
    for _ in range(M):
        U = int(random() * N) + 1
        Suma += g(U)
    return Suma / M * N

#### Ejemplo:
Sea $S = \sum_{i=1}^{10000} e^{1/i} = 10000\frac1{10000} \sum_{i=1}^{10000} e^{1/i} = 10000E[g(X)]$

In [69]:
import numpy as np

def g(U):
    return np.exp(1 / U)

print(MonteCarlo(g,10000,100))

10004.139597715299


In [37]:
#Sumatoria original para comparar
suma = 0
for i in range(10000):
    suma += g(i+1)
print(suma)

10010.865744767296


### Generación de una variable aleatoria geométrica
Si $X$ es una variable con distribución geométrica con probabilidad de éxito $p$ tiene una probabilidad de masa dada por $p_i=P(X=i)=pq^{i-1}, \quad i\ge1, \quad q=(1-p),$ y función de distribución acumulada $F_X(j-1)=P(X\le j-1)=1-P(X > j-1)=1-q^{j-1}$.

Aplicando el método de la inversa,

$X$
genera
$
j \Leftrightarrow \\
p_0+...+p_{j-1} \le U < p_o+...+p_j \Leftrightarrow \\
1-q^{j-1} \le U < 1-q^j \Leftrightarrow \\
-q^{j-1} \le U-1 < -q^j \Leftrightarrow \\
q^j < 1-U \le q^{j-1} \Leftrightarrow \\
jLog(q) <  Log(1-U) \le (j-1)Log(q) \Leftrightarrow \\
j <  \frac{Log(1-U)}{Log(q)} \le j-1 \Leftrightarrow \quad \{ q<1 \Rightarrow Log(q)<1 \} \\
j-1 \le  \frac{Log(1-U)}{Log(q)} < j \Leftrightarrow \\
j-1 = \lfloor \frac{Log(1-U)}{Log(q)} \rfloor \Leftrightarrow \\
j = \lfloor \frac{Log(1-U)}{Log(q)} \rfloor +1 \\
$



In [ ]:
def Geometrica(p):
    return (int(np.log(1-random())/np.log(1-p)) + 1)

### Generación de una variable aleatoria Bernoulli

In [ ]:
def Bernoulli(p):
    U=random()
    if U < p: return 1
    else:     return 0

#### Generar una secuencia de variables aleatorias independientes Bernoulli
Para esto podemos usar una variable aleatoria geométrica con probabilidad de éxito $p$, $Y\thicksim Geom(p)$, y definir $X_1,...,X_N \thicksim Bernoulli(p)$ tal que $Y=j$ equivale a $X_1 = ... = X_{j-1} = 0$ y $X_j = 1$. El algoritmo para generar una secuencia de variables aleatorias independientes Bernoulli es el siguiente:

In [ ]:
'''
N: Cantidad de variables Bernoulli a generar
'''
def SecuenciaDeBernoulli(p,N):
    X = [0] * N
    j = Geometrica(p) -1
    while j < N:
        X[j] = 1
        j += Geometrica(p)

### Generación de una variable aleatoria Poisson
La función de probabilidad de masa de una variable aleatoria Poisson de razón $\lambda$ está dada por
$\\ p_i=P(X=i)=e^{-\lambda}\frac{\lambda^i}{i!}, \quad i=0,1,...\\$
Luego, podemos notar que las probabilidades cumplen una relación de recurrencia:
$\\p_0 = e^{-\lambda}, \quad p_i = \frac{\lambda}{i}p_{i-1}, \quad i=1,2,...$


In [ ]:
def Poisson(lamda):
    U = random()
    i = 0; p = np.exp(-lamda)
    F = p
    while U >= F:
        i += 1
        p += lamda / i
        F = F + p
    return i

Una forma de optimizar este algoritmo es comenzar por analizar el valor mas probable. El valor máximo de las probabilidades es $p_{\lfloor λ\rfloor}$, es decir, que los valores cercanos a λ son los mas probables. Así, se puede optimizar el número de comparaciones buscando a partir del valor $I = \lfloor λ\rfloor$, y luego de manera ascendente o descendente según el valor de la variable uniforme U generada:

In [ ]:
def PoissonMejorada(lamda):
    p = np.exp(-lamda); F = p
    for j in range(1, int(lamda) + 1):
        p *= lamda / j
        F += p
    U = random()
    if U >= F:
        j = int(lamda) + 1
        while U >= F:
            p *= lamda / j; F += p
            j += 1
        return j - 1
    else:
        j = int(lamda)
        while U < F:
            F -= p; p *= j/lamda
            j -= 1
        return j+1

De esta manera, el promedio de búsquedas se reduce a $1+E[|X-\lambda|]$. En particular, si $\lambda >> 1$, la distrubución de Poisson se aproxima a una normal con media y varianza $\lambda$, por lo cual $\frac{X-\lambda}{\sqrt\lambda}$ se aproxima a una normal estándar, y por lo tanto, $1+E[|X-\lambda|]$, puede estimarse como:
$\\1+\sqrt{\lambda}E[\frac{|X-\lambda|}{\sqrt{\lambda}}]=1+\sqrt{\lambda}E[|Z|]=1+0.798\sqrt{\lambda}\\$
que mejora el promedio en la versión anterior $1+\lambda$.

### Generación de una variable aleatoria binomial
Si $X\thicksim B(n,p)$ es una variable con distribución binomial con función de masa de probabilidades 
$\\p_i=P(X=i)=\frac{n!}{i!(n-1)!}p^i(1-p)^{n-i}, \quad i=0,1,...,n.\\$
En este caso, la fórmula recursiva para las probabilidades está dada por:
$\\ p_0=(1-p)^n, \quad p_{i+1}=\frac{n-i}{i+1}\frac{p}{1-p}p_i, \quad 0\le i<n\\$
Si se aplica directamente el método de la transformada inversa, el algoritmo resulta:

In [6]:
def Binomial(n,p):
    c = p / (1 - p)
    prob = (1 - p) ** n
    F = prob; i=0
    U = random()
    while U >= F:
        prob *= c * (n-i) / (i+1)
        F += prob
        i += 1
    return i    

#### Alternativas:
Como podemos ver, el número de comparaciones que realiza este algoritmo es el valor generado más uno, es decir, $$E[\text{número de comparaciones}] = 1+E[X] = 1+E[B(n,p)] = 1+np.$$ Una forma de optimizar este algoritmo es comenzar por analizar el valor más probable, es decir, $j=\lfloor np\rfloor$, y luego de manera ascendente o descendente según el valor de la variable uniforme U generada, mejorando el promedio de comparaciones a $1+0.798\sqrt{np}$.

Otra alternativa para generar una variable aleatoria binomial es pensarla como la suma de $n$ variables aleatorias independientes Bernoulli con probabilidad de éxito $p$, es decir, $X = X_1+...+X_n$, donde $X_i\thicksim Bernoulli(p)$.

## Método de aceptación y rechazo (método de rechazo)
Supone el conocimiento de un método para la generación de otra variable aleatoria $Y$ que cumpla con las siguientes propiedades:
* Si $P(X=x_j)>0$, entonces $P(Y=x_j)>0$, para todo $X_j$ en el rango de $X$.
* Existe una constante $s$ tal que $\frac{P(X=x_j)}{P(Y=y_j)}\le c$, para todos los $x_j$ tal que $P(X=x_j)>0$.

Si denotamos $p_j = P(X=x_j)$ y $q_j = P(Y=x_j)$, entonces de la segunda propoiedad vemos que:
$$\sum_{j\ge1}p_j\le c\times\sum_{j\ge1}q_j\le c.$$

Como el miembro izquierdo de la desigualdad es 1, se tiene que la constante $c$ es siempre
mayor o igual a 1. Además la igualdad se daría solo en el caso en que $X$ e $Y$ tienen la misma
distribución de probabilidad, en cuyo caso ya se tendría un método para generar $X$. Asumimos
entonces que $c > 1$, y por lo tanto $\frac1c < 1$.

In [ ]:
'''
GenerarY: Función que genera la variable aleatoria Y
p: arreglo tal que p[i] = p_i
p: arreglo tal que q[i] = q_i
c: constante tal que p_i/q_i <= c
'''
def MetodoAceptacionRechazo(GenerarY,p,q,c):
    while True:
        Y = GenerarY()
        U = random()
        if  U < p[Y] / (c * q[Y]):
            return Y

Con esta implementación, $$P(\text{aceptar en determinada iteración } Y) $$
$$P(\text{aceptar } Y=x_1 \text{ ó aceptar } Y=x_2 \text{ ó aceptar } Y=x_n \text{ ó ...}) $$
$$\sum_{j\ge1}P(\text{aceptar } Y=x_j) $$
$$\sum_{j\ge1}P(\text{generar } Y=x_j \cap U<\frac{p_j}{c*q_j}) $$
$$\sum_{j\ge1}P(\text{generar } Y=x_j)P(U<\frac{p_j}{c*q_j}) $$
$$\sum_{j\ge1} q_j\frac{p_j}{c*q_j} = \frac1c\sum_{j\ge1}p_j = \frac1c.$$
Así, $P(\text{rechazar } Y) = 1 - \frac1c$, y
$$P(\text{generara } x_j) =$$
$$\sum_{k=1}^\infty P(\text{rechazar } Y \ k-1 \text{ veces}\cap\text{aceptar } Y \text{ en la } k\text{-ésima iteración})=$$
$$\sum_{k=1}^\infty (1-1/c)^{k-1}P(\text{generar } Y=x_j \cap U<\frac{p_j}{c*q_j})=$$
$$\sum_{k=1}^\infty (1-1/c)^{k-1}q_j \frac{p_j}{c*q_j}=$$
$$\sum_{k=1}^\infty (1-1/c)^{k-1} \frac{p_j}{c}=$$
$$p_j\sum_{k=1}^\infty (1-1/c)^{k-1} \frac1{c}= \\
    \text{\{ Probabilidad de masa de una v.a. geométrica\}}$$
$$p_j$$ 


In [ ]:
def MetodoAceptacionRechazo(GenerarY,p,q,c):
    Y = GenerarY()
    U = random()
    while U >= p(Y) / (c * q(Y)):
        Y, U = GenerarY(), random() # rechazo
    return Y # aceptación

#### Ejemplo:
Dado $X\in\{1,...,10\}$ uba variable aleatoria con función de probabilidad $p_1 = 0.11, p_2 = 0.12, p_3 = 0.09, p_4 = 0.08, p_5 = 0.12, p_6 = 0.10, p_7 = 0.09, p_8 = 0.09, p_9 = 0.10, p_{10} = 0.10$. 

Si se usa el método de la transformada inversa (mejorada), tenemos que
$E[\text{número de comparaciones}] = (1+2)\times 0.12 + 3\times 0.11 + (4+5+6)\times 0.10 + (7+8)\times 0.09 + 9\times 0.17 = 5.07.$

Si se usa el método de aceptación y rechazo, definimos $Y\thicksim\mathcal{U}\{1,10\}$, $P(Y=y_j)=0.1$. Entonces $$\frac{P(X=x_j)}{P(Y=y_j)} = \frac{p_j}{0.1} \le 1.2=c.$$ Entonces $E[\text{número de iteraciones}] = E[geom(1/c)] = c = 1.2$

## Método de composición
Se utiliza cuando tenemos una variable aleatoria discreta $X$ que, cuando toma un valor,  su probabilidad se escribe en términos de probabilidades de otras variables aleatorias $X_1,...,X_n$ que sabemos generar. Es decir,
$$P(X=x_j)=\alpha_1 P(X_1 = x_j) + \alpha_2 P(X_2 = x_j) + ... + \alpha_n P(X_n = x_j),$$
con $\alpha_1 + \alpha_2 + ... + \alpha_n = 1$, y $\alpha_1\ge 0$, para todo $x_j \in X$. 

Podemos ver que si tomamos $U\thicksim\mathcal{U}(0,1)$, entonces $P(U<\alpha_1) = \alpha_1$, y $P(\alpha_1+...+\alpha_{n-1}\le U < \alpha_1+...+\alpha_n) = \alpha_n.$ De esta manera, $$P(\text{generar }x_j)=P(X=x_j)=\\ P(U<\alpha_1 \cap X_1=x_j) + ... + P(\alpha_1+...+\alpha_{n-1}\le U < \alpha_1+...+\alpha_n \cap X_n=x_j) =\\ \alpha_1P(X_1=x_j) + ... + \alpha_nP(X_n=x_j).$$



In [38]:
'''
alpha: arreglo que contiene los valores de alpha, tal que alpha[i]=alpha_{i+1}
GenerarX: Función tal que GenerarX(i) genera X_i
'''
def MetodoComposicion(alpha,GenerarX):
    U = random()
    i = 0        # Modificado por empezar el arreglo en 0
    F = alpha[i] # alpha_1
    while U >= F:
        i += 1
        F+= alpha[i] 
    return GenerarX(i+1)

## Método del alias (Ver xd)
Es una aplicación del método de composición para generar variables aleatorias discretas que toman los valores $1,2,...n.$ Requiere construir en primer lugar los llamados *alias*, pero una vez completada este etapa, la simulación en sí es muy simple. La distribución de la variable $X$, $F_X$, se describe como una composición equiprobable de $n-1$ variables aleatorias:
$$F_X(x)=\frac1{n-1}F_1(x)+...+\frac1{n-1}F_{n-1}(x),$$
donde $F_j(x)$ es la distribución de una variable $X_j$, que toma a lo sumo dos valores posibles.

**Proposición:** Sea $P(X=j)=p_j,1\le j\le n$ una función de probabilidad de masa, con $n>1$. Entonces:
1. Existe $j$, $1\le j\le n$, tal que $p_j<\frac1{n-1}$.
2. Dado $j$ como en 1, existe $i \ne j$ tal que $p_i+p_j\ge\frac1{n-1}$.

    **Demostración:** Supongamos que no existe tal $j$. Entonces, $\sum_{j=1}^np_j\ge n\frac1{n-1}>1$, lo cual es una contradicción. Por otro lado, dado $j$ como en 1, debe existir $i$ que cumpla la segunda propiedad, ya que si no existiera, entonces $1-p_j=\sum_{i\ne j}p_i<(n-1)(\frac1{n-1}-p_j)=1-(n-1)p_j$. Esto es, $p_j>(n-1)-p_j$, lo cual solo es valido si $n=1$.

El procedimiento es como sigue. En primer lugar, notemos que si se multiplican todas las probabilidades por $n − 1$, entonces su suma es igual a $n − 1$:
$$(n − 1)p1, (n − 1)p2, , . . . , (n − 1)pn. (*)$$
La idea es distribuir estos $n − 1$ valores dados como probabilidades de masa de $n − 1$
variables aleatorias, de modo que estas variables tomen a lo sumo dos valores distintos.
Si se tuviera una variable $X$ que toma valores en $\{1, 2, . . . , n\}$ con probabilidad de masa $p1, p2, . . . , pn$, entonces se eligen $j$ e $i$ como en la proposición, y se define $X$ de la siguiente manera:
$$ X_1 = \begin{cases}
    j & \text{con probabilidad } (n-1)p_j\\
    i & \text{con probabilidad } 1-(n-1)p_j\\
\end{cases}$$

Notemos que $X_1$ toma todo el peso del valor $j$, y parte del peso del valor $i$. Digamos que de
la lista (*) de valores se han utilizado $p_j$ y una ”parte” de $p_i$, de modo que suman 1. Si se
restan estos dos valores de la lista(*), la suma total es igual a $n − 2$.
Es decir, si dividimos por $n − 2$ se tendría nuevamente una variable aleatoria $\bar{X}$ con valores en $\{1, 2, . . . , n\} − \{j\}$, que tiene probabilidades
$$P(\bar{X}=k)=\frac1{n-2}(n-1)p_k, \quad k\ne j,i.$$
y $P(\bar{X}=i)$ es tal que la suma de todas las probabilidades es 1.
Ahora se eligen nuevos índices $j$ e $i$ con las propiedades dadas por la Proposición, y se construye una variable $X_2$, cambiando $n$ por $n − 1$. El procedimiento sigue hasta que solo quedan a lo sumo dos valores por generar.


In [ ]:
'''
Ejemplo de X una v.a. que toma valores en {1,2,3,4} con
p1=7/16, p2=1/4, p3=1/8, p4=3/16
'''
def MetodoAlias():
    I = int (random() * 3)
    V = random()
    if I == 0:
        if V < 5/8: return 1
        else: return 3
    elif I == 1:
        if V < 9/16: return 4
        else: return 2
    else: ##I == 2
        if V < 11/16: return 1
        else: return 2

## Método de la urna
Sea $X \in \{x_1, x_2, . . . , x_n\}$ una variable aleatoria con función de probabilidad de masa $p_1, p_2, . . . , p_n$.
El método consiste en considerar un valor $k ∈ \N$ tal que $k p_j$ sea entero, para todo $j \in \{1, n\}$. Ahora se considera un arreglo $A$ de $k$ posiciones, y se almacena cada valor $i$ en $k p_i$ posiciones del arreglo.
De esta manera, el algoritmo simplemente selecciona una posición al azar del arreglo, con probabilidad \frac1k, y devuelve el valor en dicha posición.

In [40]:
def MetodoUrna(A,k):
    return A[int(random() * k)]

#### Ejemplo:
Si $X \in \{1,2,3,4\}$, tal que $p_1 = 0.25, p_2 = 0.30, p_3 = 0.14, p_4 = 0.31$, entonces se puede tomar $k = 100$, tal que $k p_1 = 50, k p_2 = 30, k p_3 = 14, k p_4 = 31$.

Luego, el algoritmo es el siguiente:

In [68]:
def ejemplo():
    A = [1]*25 + [2]*30 + [3]*14 + [4]*31    
    return MetodoUrna(A,100)

print(ejemplo())

2


Luego, una mejora podría ser seperar de las probabilidades los centéimos, de manera que,
$$p_1 = 0.25 = 0.2 + 0.05,$$
$$p_2 = 0.30 = 0.3 + 0.00,$$
$$p_3 = 0.14 = 0.1 + 0.04,$$
$$p_4 = 0.31 = 0.3 + 0.01;$$
y creamos dos arreglos,
$$A = [1,1,2,2,2,3,4,4,4], \text{ con probabilidad 0.9 = 0.2 + 0.3 + 0.1 + 0.3}$$
$$B = [1,1,1,1,1,3,3,3,3,4], \text{ con probabilidad 0.1 = 0.05 + 0.04 + 0.01}.$$
Y, entonces, el algoritmo es el siguiente:

In [ ]:
def EjemploUrnaMejorada():
    A = [1]*2 + [2]*3 + [3]*1 + [4]*3
    B = [1]*5 + [2]*0 + [3]*4 + [4]*1
    if random()<0.9:
        return A[int(9*random())]
    else:
        return B[int(10*random())]